In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 72kB/s 
     |████████████████████████████████| 61kB 24.0MB/s 
     |████████████████████████████████| 419kB 48.5MB/s 
     |████████████████████████████████| 3.0MB 38.6MB/s 


### Get Model files and Datasets using Google storage APIs

In [0]:
# from google.colab import files
# files.upload()

In [3]:
!ls -al

total 16
drwxr-xr-x 1 root root 4096 May 15 16:23 .
drwxr-xr-x 1 root root 4096 May 19 03:37 ..
drwxr-xr-x 1 root root 4096 May 16 16:08 .config
drwxr-xr-x 1 root root 4096 May 15 16:23 sample_data


In [4]:
!wget https://storage.googleapis.com/columbia_applied_deep_learning/demand_forecasting.zip  \
    -O ./demand_forecasting.zip

--2019-05-19 03:38:38--  https://storage.googleapis.com/columbia_applied_deep_learning/demand_forecasting.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3361698 (3.2M) [application/zip]
Saving to: ‘./demand_forecasting.zip’

./demand_forecastin 100%[===================>]   3.21M  --.-KB/s    in 0.03s   

2019-05-19 03:38:38 (128 MB/s) - ‘./demand_forecasting.zip’ saved [3361698/3361698]



In [0]:
import zipfile
local_zip = './demand_forecasting.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()

### Imports

In [6]:
import pandas as pd
import numpy as np
import argparse
import os
import platform
import datetime, time
import datetime as dt
import warnings
from IPython.core.display import display, HTML
import tensorflow as tf
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, Input, LSTM, TimeDistributed,RepeatVector, Flatten
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,RemoteMonitor
import sys
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from tensorflow.keras.models import load_model

plt.style.use('default')
plt.style.use('seaborn-deep')

sns.set(style="whitegrid")
sns.set_context("talk")

%matplotlib inline
init_notebook_mode(connected=True)

In [7]:
tf.__version__

'2.0.0-alpha0'

In [0]:
checkpoint_dir = '/content/'
model_filename = 'model1_th.hdf5'
data_filename = 'Product1-Dataset-th-1year.csv'
train_model = False

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
def plotly_df(df, title='', annotations=None, forecast=False):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode='lines+markers')
    if forecast:
      xaxis = dict(title='Time Steps',type='date', rangeslider=dict(visible=False))
    else:
      xaxis = dict(title='Time Steps',type='date', rangeslider=dict(visible=True))
    
    data = [go.Scatter(y=df[c], name=c,  **common_kw) for c in df.columns]
    layout = dict(title=title, showlegend=True, annotations=annotations, xaxis=xaxis, )
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

In [0]:
def create_timeseries_dataset(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var_lag%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var_roll%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var_roll%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [0]:
def mean_absolute_percentage_error(y_pred, y_true): 
    return np.mean(np.abs((y_pred - y_true)) / (y_pred+ 1e-7)) * 100

### Read and Explore Dataset

In [0]:
df_raw = pd.read_csv(filepath_or_buffer=data_filename, header=0, names=None, usecols=['date', 'sale_amt'], parse_dates=[0])

In [0]:
df_raw.index = df_raw.date
df_raw.drop(['date'], inplace=True, axis=1)

In [15]:
configure_plotly_browser_state()
plotly_df(df_raw, title='Timeseries')

Note that during the first 30 days, there is no defined pattern. These are the beginning days of business and can be ignored for sales prediction.

In [0]:
df_raw =df_raw[30:]

In [17]:
configure_plotly_browser_state()
plotly_df(df_raw, title='Timeseries')

### Baseline Metrics (7-day moving average )

In [0]:
train, val, test = df_raw['sale_amt'].values[:-14], df_raw['sale_amt'].values[-14:], df_raw['sale_amt'].values[-14:]

In [0]:
def baseline():
  # Moving average of last few days
  
  train_base = df_raw['sale_amt'][:-14]
  rolling_mean = train_base.rolling(window=7).mean()
  predictions = list()
  predictions = rolling_mean.values[-14:]
  
  for i in range(len(val)):    
    print('>Predicted=%.f, Expected=%.f' % (predictions[i], val[i]))

  df_out = pd.DataFrame(df_raw['sale_amt'][-14:])
  df_out.columns=['Original']
  df_out['Predicted'] = predictions
  
  
  return df_out

In [20]:
df_out = baseline()

>Predicted=1608278, Expected=1171181
>Predicted=1528046, Expected=3167045
>Predicted=1483736, Expected=1117314
>Predicted=1501021, Expected=926865
>Predicted=1466179, Expected=1107539
>Predicted=1443577, Expected=1165756
>Predicted=1443577, Expected=0
>Predicted=1450894, Expected=1367183
>Predicted=1456166, Expected=2858611
>Predicted=1426546, Expected=1113747
>Predicted=1431703, Expected=1194197
>Predicted=1395973, Expected=1104006
>Predicted=1389219, Expected=1714005
>Predicted=1389219, Expected=0


In [21]:
configure_plotly_browser_state()
plotly_df(df_out, title='Baseline Predictions i.e., 7-day moving average for last 14 days', forecast=True)

In [22]:
rmse = math.sqrt(mean_squared_error(df_out['Original'].values, df_out['Predicted'].values))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 840058.865


In [23]:
mape = mean_absolute_percentage_error(np.array(df_out['Predicted'].values),np.array(df_out['Original'].values))
print ('Test MAPE: %.3f' %mape)

Test MAPE: 44.713


In [24]:
err = (abs(sum(df_out['Predicted'].values) - sum(df_out['Original'].values)) / (sum(df_out['Original'].values))) *100
print ('Prediction Error: %.2f%%' %err)

Prediction Error: 13.36%


### Statefull LSTM Model

In [0]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [0]:
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train_scaled = scaler.transform(train)
	# transform test
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

In [0]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

In [0]:
def fit_lstm(train, batch_size, nb_epoch, neurons, test = None, load_model = False):
    
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    if test.any():
        X_test, y_test = test[:, 0:-1], test[:, -1]
        X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
                
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), 
                                stateful=True, return_sequences=True))
    model.add(tf.keras.layers.LSTM(neurons ,stateful=True))
    model.add(tf.keras.layers.Dense(1))
    model.compile('adam', loss='mse')
    
    if load_model:
        pass 
    
    c = [
          ModelCheckpoint(checkpoint_dir + model_filename, save_best_only=True,monitor='val_loss', mode='min', verbose=1, period=1),
          EarlyStopping(monitor='val_loss', min_delta=0, patience=150, verbose=1),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=55, min_lr=0.0001, verbose=1)]
    
    
    for i in range(nb_epoch):
        print(i)
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False, validation_data = (X_test, y_test) , callbacks = c)
        model.reset_states()
        
    return model

In [0]:
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

In [0]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        #print(dataset[i])
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [0]:
def inverse_difference(history, yhat, interval=1):
  return yhat + history[-interval]

**Data processing**

In [0]:
# transform data to be stationary
raw_values = df_raw['sale_amt'].values
diff_values = difference(raw_values, 1)

In [0]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

In [0]:
# split data into train and test-sets
train, test = supervised_values[:len(supervised_values)-14], supervised_values[len(supervised_values)-14:]

In [0]:
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

In [0]:
if train_model:
  # fit the model
  lstm_model = fit_lstm(train_scaled, 1, 1500, 30, test_scaled)

In [0]:
# from google.colab import files
# files.download(model_filename)

In [0]:
# Load the Model file and do the inference

def forecast(train_scaled=train_scaled,test_scaled=test_scaled):  
  model_from_file = load_model(checkpoint_dir + model_filename)

  # forecast the entire training dataset to build up state for forecasting
  train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
  model_from_file.predict(train_reshaped, batch_size=1)

  # walk-forward validation on the test data
  predictions = list()
  for i in range(len(test_scaled)):
      # make one-step forecast
      X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
      yhat = forecast_lstm(model_from_file, 1, X)
      # invert scaling
      yhat = invert_scale(scaler, X, yhat)
      # invert differencing
      yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
      # store forecast
      predictions.append(yhat)
      expected = raw_values[len(train) + i + 1]
#       print('day=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))  
      
  
  df_out = pd.DataFrame(df_raw[-14:]['sale_amt'])
  df_out.columns=['Original']
  df_out['Predicted'] = predictions
  
  
  return df_out

In [39]:
df_pred = forecast()

W0519 03:38:45.503514 139874472458112 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f36986f24a8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0519 03:38:46.168810 139874472458112 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f36986f6160>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [40]:
configure_plotly_browser_state()
plotly_df(df_pred, title='Predictions', forecast = True)

In [41]:
configure_plotly_browser_state()
plotly_df(df_raw, title='Timeseries')

In [42]:
rmse = math.sqrt(mean_squared_error(df_pred['Original'].values, df_pred['Predicted'].values))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 219359.992


In [43]:
mape = mean_absolute_percentage_error(np.array(df_pred['Predicted'].values),np.array(df_pred['Original'].values))
print ('Test MAPE: %.3f' %mape)

Test MAPE: 9.398


In [44]:
err = (abs(sum(df_pred['Predicted'].values) - sum(df_pred['Original'].values)) / (sum(df_pred['Original'].values))) *100
print ('Prediction Error: %.2f%%' %err)

Prediction Error: 0.28%


### Prediction on 2 year dataset

In [0]:
data_filename = 'Product1-Dataset-th-2year.csv'

In [0]:
df_raw = pd.read_csv(filepath_or_buffer=data_filename, header=0, names=None, usecols=['date', 'sale_amt'], parse_dates=[0])

In [0]:
df_raw.index = df_raw.date
df_raw.drop(['date'], inplace=True, axis=1)

In [0]:
df_raw =df_raw[30:]

In [49]:
configure_plotly_browser_state()
plotly_df(df_raw, title='Timeseries')

In [0]:
# transform data to be stationary
raw_values = df_raw['sale_amt'].values
diff_values = difference(raw_values, 1)

In [0]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

In [0]:
# split data into train and test-sets
train, test = supervised_values[:len(supervised_values)-14], supervised_values[len(supervised_values)-14:]

In [0]:
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

In [0]:
# Load the Model file and do the inference

def forecast(train_scaled=train_scaled,test_scaled=test_scaled):  
  model_from_file = load_model(checkpoint_dir + model_filename)

  # forecast the entire training dataset to build up state for forecasting
  train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
  model_from_file.predict(train_reshaped, batch_size=1)

  # walk-forward validation on the test data
  predictions = list()
  for i in range(len(test_scaled)):
      # make one-step forecast
      X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
      yhat = forecast_lstm(model_from_file, 1, X)
      # invert scaling
      yhat = invert_scale(scaler, X, yhat)
      # invert differencing
      yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
      # store forecast
      predictions.append(yhat)
      expected = raw_values[len(train) + i + 1]
#       print('day=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))  
      
  
  df_out = pd.DataFrame(df_raw[-14:]['sale_amt'])
  df_out.columns=['Original']
  df_out['Predicted'] = predictions
  
  
  return df_out

In [55]:
df_pred = forecast()

W0519 03:38:51.601475 139874472458112 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f366e145dd8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0519 03:38:51.682123 139874472458112 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f366e1450f0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [56]:
configure_plotly_browser_state()
plotly_df(df_pred, title='Predictions', forecast = True)

In [57]:
rmse = math.sqrt(mean_squared_error(df_pred['Original'].values, df_pred['Predicted'].values))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 343458.115


In [58]:
mape = mean_absolute_percentage_error(np.array(df_pred['Predicted'].values),np.array(df_pred['Original'].values))
print ('Test MAPE: %.3f' %mape)

Test MAPE: 16.202


In [59]:
err = (abs(sum(df_pred['Predicted'].values) - sum(df_pred['Original'].values)) / (sum(df_pred['Original'].values))) *100
print ('Prediction Error: %.2f%%' %err)

Prediction Error: 10.51%
